In [9]:
import pandas as pd
import h5py
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from selfpeptide.utils.model_utils import *
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader
from selfpeptide.utils.data_utils import PreTokenized_HumanPeptidesDataset

In [3]:
sns_model = load_sns_model("/home/gvisona/Projects/SelfPeptides/trained_models/sns_model")

In [7]:
config = {"immunogenicity_df": "/home/gvisona/Projects/SelfPeptides/processed_data/Immunogenicity/Processed_TCell_IEDB_Beta_noPrior.csv",
          "pseudo_seq_file": "/home/gvisona/Projects/SelfPeptides/data/NetMHCpan_pseudoseq/MHC_pseudo.dat",
          "dhlap_df": "/home/gvisona/Projects/SelfPeptides/processed_data/Immunogenicity/DHLAP_immunogenicity_data.csv",
          "seed": 0,
          "hla_filter": None,
          "beta_prior": "uniform",
          "hdf5_dataset": "/home/gvisona/Projects/SelfPeptides/processed_data/Self_nonSelf/pre_tokenized_peptides_dataset.hdf5",
          "batch_size": 16,
          "test_run": False
}


print("Evaluating Cosine Centroid for Human Peptides")
p_dset = PreTokenized_HumanPeptidesDataset(
    config["hdf5_dataset"], test_run=config["test_run"])
p_loader = DataLoader(
    p_dset, batch_size=config["batch_size"], drop_last=False)


Evaluating Cosine Centroid for Human Peptides


In [11]:
device="cpu"
ref_human_peptides_vector = None
n_peptides = len(p_dset)
for ix, peptides in tqdm(enumerate(p_loader)):
    if torch.is_tensor(peptides):
        peptides = peptides.to(device)
    projections, embeddings = sns_model(peptides)

    projections = projections / projections.norm(dim=1)[:, None]
    if ref_human_peptides_vector is None:
        ref_human_peptides_vector = torch.sum(projections.detach(), dim=0)
    else:
        ref_human_peptides_vector += torch.sum(projections.detach(), dim=0)
    break
# ref_human_peptides_vector /= n_peptides
# ref_human_peptides_vector = (ref_human_peptides_vector / 
#                             ref_human_peptides_vector.norm())
# model.human_peptides_cosine_centroid = ref_human_peptides_vector
# torch.save(model.state_dict(), checkpoint_path)


0it [00:00, ?it/s]

Training complete!


In [12]:
ref_human_peptides_vector

tensor([ 2.3808,  0.1968,  1.3967,  1.5287, -0.3774,  3.4352,  1.7573,  0.7584,
        -0.4940,  0.4538,  0.9627, -0.0541,  0.1341, -4.5653,  2.5507, -5.5505,
        -2.1186,  0.6783, -2.3996,  0.0415, -2.7242, -1.8089, -0.7070,  3.3590,
         8.7504, -1.3897, -0.1217, -0.5307,  1.8615, -0.4715, -0.6142, -5.0446])